## couples each vbo to a grid



In [19]:
from shapely.geometry import asPoint
import geopandas as gpd
import pandas as pd
import time
import shapely
import matplotlib.pyplot as plt
import os

In [2]:
def VBOCleaner(data):
    print(data.shape)
    
    # only keep objects with id
    data = data[data["identificatie"].notna()]
    print(data.shape)
    
    # only keep objects which are currently in use # CHECK of dit de juiste zijn
    data = data[data.verblijfsobjectStatus.isin(["Verblijfsobject gevormd", 
                                                 "Verblijfsobject in gebruik",
                                                 "Verblijfsobject in gebruik (niet ingemeten)"])]
    data = data[data.pos.notna()]
    print(data.shape)
    
    data["begindatumTijdvakGeldigheid"] =  pd.to_datetime(data['begindatumTijdvakGeldigheid'], format='%Y%m%d%H%M%S%f', errors='coerce')
    data["einddatumTijdvakGeldigheid"] =  pd.to_datetime(data['einddatumTijdvakGeldigheid'], format='%Y%m%d%H%M%S%f', errors='coerce')
    
    def toPoint(pos):
        return asPoint([float(i) for i in pos.split()[:2]])
    data["pos"] = data.pos.apply(toPoint)
    
    return data

In [3]:
vbo = pd.read_csv("VBO.csv", sep = ";", encoding = "utf-8", na_values = "NaN", on_bad_lines = "warn")
vbo = VBOCleaner(vbo)


(19501545, 7)
(18918593, 7)
(18386142, 7)


In [20]:
pand = pd.read_csv("pand_cleaned.csv", sep = ";")


C:\Users\Gebruiker\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# # Grid is full eindhoven grid
# grid = pd.read_csv("Nijmegen_grid.csv")
# grid['geometry'] = grid.geometry.apply(shapely.wkt.loads)
# grid = gpd.GeoDataFrame(grid)

In [22]:
def couple_grid_vbo(PATH, vbo):
    
    vbopoints = gpd.GeoDataFrame(vbo, geometry = "pos")
    for filename in os.listdir(PATH):
        
        t0 = time.time()
        name = filename.strip("_grid.csv")
        
        grid = pd.read_csv(PATH + filename) # get grid
        grid['geometry'] = grid.geometry.apply(shapely.wkt.loads) # load geometry as shape
        grid = gpd.GeoDataFrame(grid)
        print(grid.shape)
        # join each cbs row with a vbo that contains a vbo point
        vbogrid = gpd.tools.sjoin(vbopoints, grid, op = "within")
        print("{} done. time:{}".format(name,time.time() - t0))

        vbogrid.to_csv("../Data/presnap/vbo/" + name + "_vbo.csv")

In [23]:
coupled = couple_grid_vbo("../Data/presnap/grid/", vbo)

(68355, 3)
Almere done. time:1489.4407768249512
(19324, 3)
Breda done. time:15.673068761825562


In [ ]:
# plot example

plt.plot(*cbs[cbs["C28992R100"] == pointInPolys.iloc[0].C28992R100].geometry.iloc[0].exterior.xy)
plt.scatter(point.x, point.y)